In [1]:
# IMPORTS
import numpy as np
import pandas as pd
import requests
from io import StringIO
from fredapi import Fred
from dotenv import load_dotenv
import os
#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# measure time for ML HyperParams search
import time
from datetime import date

# ML models and utils
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'
     


In [2]:
url = f"https://stockanalysis.com/list/biggest-companies/"
headers = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/58.0.3029.110 Safari/537.3'
    )
}

try:
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()

    # Wrap HTML text in StringIO to avoid deprecation warning
    # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
    html_io = StringIO(response.text)
    table = pd.read_html(html_io)
    df = table[0]
    symbols = df["Symbol"].to_list()[:100]

    if not table:
        raise ValueError(f"No table found")

    
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except ValueError as ve:
    print(f"Data error: {ve}")
except Exception as ex:
    print(f"Unexpected error: {ex}")
symbols[symbols.index('BRK.B')] = 'BRK-B'
print(symbols)


['NVDA', 'MSFT', 'AAPL', 'GOOGL', 'AMZN', 'META', 'AVGO', 'TSLA', 'BRK-B', 'TSM', 'JPM', 'WMT', 'V', 'LLY', 'ORCL', 'MA', 'NFLX', 'XOM', 'JNJ', 'COST', 'HD', 'PLTR', 'BAC', 'ABBV', 'PG', 'CVX', 'SAP', 'KO', 'ASML', 'GE', 'TMUS', 'BABA', 'UNH', 'CSCO', 'AMD', 'WFC', 'PM', 'TM', 'AZN', 'NVS', 'NVO', 'MS', 'CRM', 'ABT', 'IBM', 'LIN', 'HSBC', 'MCD', 'AXP', 'GS', 'RTX', 'SHEL', 'MRK', 'DIS', 'BX', 'T', 'CAT', 'UBER', 'PEP', 'RY', 'TXN', 'VZ', 'BKNG', 'TMO', 'INTU', 'SHOP', 'NOW', 'BA', 'SCHW', 'BLK', 'C', 'PDD', 'MUFG', 'QCOM', 'HDB', 'GEV', 'ISRG', 'ANET', 'SPGI', 'SONY', 'ACN', 'APP', 'BSX', 'AMGN', 'UL', 'NEE', 'TJX', 'SYK', 'ADBE', 'ARM', 'DHR', 'LOW', 'PGR', 'COF', 'SPOT', 'GILD', 'PFE', 'HON', 'SAN', 'BHP']


In [3]:
tariff_stocks = ["FINV", "TSM", "LITE", "VOYA", "KO", "BUG", "WEC"]
duplicates = set(tariff_stocks).intersection(set(symbols))
for dup in duplicates: symbols.remove(dup)
print(len(symbols))

98


In [4]:
commodity_funds = ["GLD", "USO", "MOO", "WELL", "BND", "REMX", "SLV"]
world_indices = ['^GSPC', '^DJI', '^STOXX', '^GDAXI', '^FTSE', '^N225', '^VIX']
commodities = ['GC=F', 'HG=F', 'BZ=F', 'S=F', 'BTC-USD']
currencies = ["EURUSD=X", "GBPUSD=X", "JPY=X", "CNY=X"]
bonds = ['IRLTLT01DEM156N', 'IRLTLT01GBM156N', 'IRLTLT01JPM156N'] #German, UK, Japan 
tickers = symbols + tariff_stocks + commodity_funds
indices = world_indices + commodities + currencies  
macros = ['GDPPOT', 'CPILFESL', 'FEDFUNDS',
 'DGS1', 'DGS5', 'DGS10', 'CSUSHPISA'] 

In [5]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-50, month=end.month, day=end.day)

print(f'Period for indexes: {start} to {end} ')

Year = 2025; month= 8; day=26
Period for indexes: 1975-08-26 to 2025-08-26 


In [6]:
stocks_df = pd.DataFrame({'A' : []})

for i, t in enumerate(tickers): 
    print(i, t)
    historyPrices = yf.Ticker(t).history(start=start, #period = "max", 
        interval = "1d")
    if t in symbols: historyPrices['ticker_type'] = 'biggest_company'
    elif t in tariff_stocks: historyPrices['ticker_type'] = 'tariff'
    else: historyPrices['ticker_type'] = 'commodity_fund'
    historyPrices['Ticker'] = t
    historyPrices['Year']= historyPrices.index.year
    historyPrices['Month'] = historyPrices.index.month
    historyPrices['Weekday'] = historyPrices.index.weekday
    historyPrices['Date'] = historyPrices.index.date
     # historical returns
    for i in [1,3,7,30,90,365]:
         historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
    historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']
    
    # # Technical indicators
    # # SimpleMovingAverage 10 days and 20 days
    historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
    historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
    historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
    historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

    # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
    historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)
    # what we want to predict
    historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)
    
    # sleep 1 sec between downloads - not to overload the API server
    time.sleep(1)
    
    
    if stocks_df.empty:
        stocks_df = historyPrices
    else:
        stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 NVDA
1 MSFT
2 AAPL
3 GOOGL
4 AMZN
5 META
6 AVGO
7 TSLA
8 BRK-B
9 JPM
10 WMT
11 V
12 LLY
13 ORCL
14 MA
15 NFLX
16 XOM
17 JNJ
18 COST
19 HD
20 PLTR
21 BAC
22 ABBV
23 PG
24 CVX
25 SAP
26 ASML
27 GE
28 TMUS
29 BABA
30 UNH
31 CSCO
32 AMD
33 WFC
34 PM
35 TM
36 AZN
37 NVS
38 NVO
39 MS
40 CRM
41 ABT
42 IBM
43 LIN
44 HSBC
45 MCD
46 AXP
47 GS
48 RTX
49 SHEL
50 MRK
51 DIS
52 BX
53 T
54 CAT
55 UBER
56 PEP
57 RY
58 TXN
59 VZ
60 BKNG
61 TMO
62 INTU
63 SHOP
64 NOW
65 BA
66 SCHW
67 BLK
68 C
69 PDD
70 MUFG
71 QCOM
72 HDB
73 GEV
74 ISRG
75 ANET
76 SPGI
77 SONY
78 ACN
79 APP
80 BSX
81 AMGN
82 UL
83 NEE
84 TJX
85 SYK
86 ADBE
87 ARM
88 DHR
89 LOW
90 PGR
91 COF
92 SPOT
93 GILD
94 PFE
95 HON
96 SAN
97 BHP
98 FINV
99 TSM
100 LITE
101 VOYA
102 KO
103 BUG
104 WEC
105 GLD
106 USO
107 MOO
108 WELL
109 BND
110 REMX
111 SLV


In [7]:
pd.set_option('display.max_columns', None)
stocks_df.tail(50)


,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker_type,Ticker,Year,Month,Weekday,Date,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Capital Gains
925211,33.099998,33.150002,32.900002,33.029999,14883300,0.0,0.0,commodity_fund,SLV,2025,6,0,2025-06-16,1.001820,1.002428,1.017874,1.134272,1.125383,1.516529,1.014532,32.655000,31.4595,1,0.007569,22.720837,1,0.0
925212,33.630001,33.869999,33.450001,33.740002,26824900,0.0,0.0,commodity_fund,SLV,2025,6,1,2025-06-17,1.021496,1.020569,1.032120,1.144505,1.148400,1.558430,0.987552,32.885000,31.6755,1,0.012448,23.824204,0,0.0
925213,33.740002,33.750000,33.209999,33.270000,21945600,0.0,0.0,commodity_fund,SLV,2025,6,2,2025-06-18,0.986070,1.009099,0.996406,1.100927,1.146056,1.579772,1.009618,33.078000,31.8325,1,0.016231,24.597962,1,0.0
925214,32.779999,32.790001,32.570000,32.720001,25415200,0.0,0.0,commodity_fund,SLV,2025,6,4,2025-06-20,0.983469,0.990615,0.984652,1.109529,1.120548,1.552182,1.038509,33.105000,31.9430,1,0.006724,24.554755,1,0.0
925215,32.810001,33.049999,32.779999,32.959999,18661900,0.0,0.0,commodity_fund,SLV,2025,6,0,2025-06-23,1.007335,0.976882,1.000303,1.118805,1.136552,1.553984,1.042172,33.132000,32.0895,1,0.008192,24.498953,1,0.0
925216,32.549999,32.630001,32.060001,32.610001,20982400,0.0,0.0,commodity_fund,SLV,2025,6,1,2025-06-24,0.989381,0.980162,0.986388,1.094295,1.111452,1.543303,1.053358,33.054000,32.1975,1,0.017479,24.343808,1,0.0
925217,32.529999,33.009998,32.490002,33.009998,12254300,0.0,0.0,commodity_fund,SLV,2025,6,2,2025-06-25,1.012266,1.008863,1.001213,1.114074,1.120502,1.571905,1.053014,33.032000,32.3365,1,0.015753,24.170255,1,0.0
925218,33.130001,33.369999,33.000000,33.340000,15255200,0.0,0.0,commodity_fund,SLV,2025,6,3,2025-06-26,1.009997,1.011529,1.009385,1.113933,1.137496,1.592168,1.046191,33.071000,32.5045,1,0.011098,24.241867,1,0.0
925219,32.560001,32.900002,32.529999,32.619999,17596300,0.0,0.0,commodity_fund,SLV,2025,6,4,2025-06-27,0.978404,1.000307,0.966805,1.115214,1.090605,1.566763,1.047823,33.027000,32.6205,1,0.011343,23.312622,1,0.0
925220,32.720001,32.820000,32.619999,32.810001,10565300,0.0,0.0,commodity_fund,SLV,2025,6,0,2025-06-30,1.005825,0.993941,0.986174,1.108072,1.100637,1.548372,1.048766,33.011000,32.7610,1,0.006096,22.599157,1,0.0


In [8]:
stocks_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'ticker_type', 'Ticker', 'Year', 'Month', 'Weekday', 'Date',
       'growth_1d', 'growth_3d', 'growth_7d', 'growth_30d', 'growth_90d',
       'growth_365d', 'growth_future_30d', 'SMA10', 'SMA20',
       'growing_moving_average', 'high_minus_low_relative', 'volatility',
       'is_positive_growth_30d_future', 'Capital Gains'],
      dtype='object')

In [9]:
def get_growth_df(df:pd.DataFrame, prefix:str)->pd.DataFrame:
  for i in [1,3,7,30,90,365]:
    df['growth_'+prefix+'_'+str(i)+'d'] = df['Close'] / df['Close'].shift(i)
    GROWTH_KEYS = [k for k in df.keys() if k.startswith('growth')]
  return df[GROWTH_KEYS]

In [10]:
to_merge = []
for i, t in enumerate(indices): 
    print(i, t)    
    econ = yf.Ticker(t).history(start=start, #period = "max", 
                                interval = "1d")
    econ.index = econ.index.date
  
    if t not in currencies and t != '^VIX':
        to_merge.append(get_growth_df(econ, t))
    else: to_merge.append(econ.rename(columns={"Close": t+"_Close"})[
            [t+"_Close"]])
    time.sleep(1)
merged = []
merged.append(to_merge[0])
for j in range(1, len(to_merge)):
    merged.append(pd.merge(
            merged[-1],
            to_merge[j],
            left_index=True,
            right_index=True,
            how="left",
            validate="one_to_one",
        ))
econ_indices_df = merged[-1]
#econ_indices_df["Date"] = econ_indices_df.index.date
del merged
    

0 ^GSPC
1 ^DJI
2 ^STOXX
3 ^GDAXI
4 ^FTSE
5 ^N225
6 ^VIX
7 GC=F
8 HG=F
9 BZ=F
10 S=F
11 BTC-USD
12 EURUSD=X
13 GBPUSD=X
14 JPY=X
15 CNY=X


In [10]:
# # from functools import reduce
# econ_indices_df = pd.DataFrame({'A' : []})

# for i, t in enumerate(indices): 
#     print(i, t)    
#     econ = yf.Ticker(t).history(period = "max", interval = "1d")
#     econ['Ticker'] = t    
#     econ['Year']= econ.index.year
#     econ['Month'] = econ.index.month
#     econ['Weekday'] = econ.index.weekday
#     econ['Date'] = econ.index.date
#     if t != '^VIX':
#         for i in [1,3,7,30,90,252]:
#              econ['growth_'+str(i)+'d'] = econ['Close'] / econ['Close'].shift(i)
#         econ['growth_future_30d'] = econ['Close'].shift(-30) / econ['Close']
   
#     # # Technical indicators
#     # # SimpleMovingAverage 10 days and 20 days
#       # sleep 1 sec between downloads - not to overload the API server
#     time.sleep(1)

#     if econ_indices_df.empty:
#         econ_indices_df = econ
#     else:
#         econ_indices_df = pd.concat([econ_indices_df, econ], ignore_index=True)

0 ^GSPC
1 ^STOXX
2 ^GDAXI
3 ^FTSE
4 ^N225
5 ^VIX
6 GC=F
7 HG=F
8 BZ=F
9 S=F
10 BTC-USD
11 EURUSD=X
12 GBPUSD=X
13 JPY=X
14 CNY=X


In [11]:
econ_indices_df.tail(12)

,growth_^GSPC_1d,growth_^GSPC_3d,growth_^GSPC_7d,growth_^GSPC_30d,growth_^GSPC_90d,growth_^GSPC_365d,growth_^DJI_1d,growth_^DJI_3d,growth_^DJI_7d,growth_^DJI_30d,growth_^DJI_90d,growth_^DJI_365d,growth_^STOXX_1d,growth_^STOXX_3d,growth_^STOXX_7d,growth_^STOXX_30d,growth_^STOXX_90d,growth_^STOXX_365d,growth_^GDAXI_1d,growth_^GDAXI_3d,growth_^GDAXI_7d,growth_^GDAXI_30d,growth_^GDAXI_90d,growth_^GDAXI_365d,growth_^FTSE_1d,growth_^FTSE_3d,growth_^FTSE_7d,growth_^FTSE_30d,growth_^FTSE_90d,growth_^FTSE_365d,growth_^N225_1d,growth_^N225_3d,growth_^N225_7d,growth_^N225_30d,growth_^N225_90d,growth_^N225_365d,^VIX_Close,growth_GC=F_1d,growth_GC=F_3d,growth_GC=F_7d,growth_GC=F_30d,growth_GC=F_90d,growth_GC=F_365d,growth_HG=F_1d,growth_HG=F_3d,growth_HG=F_7d,growth_HG=F_30d,growth_HG=F_90d,growth_HG=F_365d,growth_BZ=F_1d,growth_BZ=F_3d,growth_BZ=F_7d,growth_BZ=F_30d,growth_BZ=F_90d,growth_BZ=F_365d,growth_S=F_1d,growth_S=F_3d,growth_S=F_7d,growth_S=F_30d,growth_S=F_90d,growth_S=F_365d,growth_BTC-USD_1d,growth_BTC-USD_3d,growth_BTC-USD_7d,growth_BTC-USD_30d,growth_BTC-USD_90d,growth_BTC-USD_365d,EURUSD=X_Close,GBPUSD=X_Close,JPY=X_Close,CNY=X_Close
2025-08-11,0.997496,1.004474,1.005373,1.032460,1.131435,1.257207,0.995461,0.995066,0.996468,1.003556,1.047276,1.125568,0.999415,1.010516,0.993676,1.005758,1.008596,1.113451,0.996626,1.006562,1.000659,1.007182,1.075500,1.357837,1.003738,0.996225,0.999661,1.042084,1.057315,1.196553,NaN,NaN,NaN,NaN,NaN,NaN,16.25,0.974993,0.992041,1.018189,1.017818,1.067900,1.649336,0.992818,1.007402,1.021476,0.879423,0.880561,1.155715,1.000601,0.996113,0.918654,0.985505,0.888993,0.796248,NaN,NaN,NaN,NaN,NaN,NaN,0.995178,1.017506,1.031802,1.011038,1.139787,2.022011,1.164795,1.344447,147.664001,7.1810
2025-08-12,1.011345,1.016681,1.033304,1.038809,1.136624,1.269305,1.010995,1.011144,1.019960,1.008251,1.052890,1.140771,1.002067,1.003370,1.003259,1.012044,1.026165,1.106625,0.997651,0.993067,1.025562,1.014848,1.106246,1.356093,1.001982,1.005164,1.008733,1.041262,1.062647,1.190732,1.021465,1.047146,1.040135,1.063944,1.199124,1.133008,14.73,0.998747,0.984884,1.000358,1.003656,1.081337,1.637044,1.019216,1.029691,1.021756,0.893126,0.937123,1.175773,0.992346,0.995333,0.949046,0.985248,0.942686,0.790720,NaN,NaN,NaN,NaN,NaN,NaN,1.012141,1.031524,1.052842,1.008872,1.160649,2.024663,1.161845,1.343526,148.125000,7.1877
2025-08-13,1.003230,1.012071,1.021586,1.043332,1.198287,1.275520,1.010429,1.016902,1.016947,1.009604,1.107935,1.153361,1.005402,1.006891,1.018960,1.019621,1.020773,1.107793,1.006694,1.000941,1.018011,1.016624,1.171685,1.366541,1.001902,1.007641,1.004042,1.044503,1.081478,1.199586,1.013027,1.053959,1.060664,1.068843,1.211298,1.134093,14.49,1.002926,0.976622,0.995347,1.003196,1.115106,1.609421,0.994122,1.005948,1.015176,0.870460,1.022237,1.162646,0.992589,0.985583,0.954479,0.949645,1.000762,0.785518,NaN,NaN,NaN,NaN,NaN,NaN,1.026388,1.033840,1.072296,1.029156,1.188920,2.035059,1.167706,1.350421,147.757996,7.1785
2025-08-14,1.000303,1.014920,1.026884,1.038719,1.274820,1.269269,0.999755,1.021289,1.018125,1.009595,1.172163,1.151677,1.005482,1.013004,1.023033,1.023392,1.031569,1.117958,1.007935,1.012298,1.022286,1.018525,1.231833,1.375961,1.001309,1.005203,1.003774,1.040121,1.139317,1.200230,0.985548,1.019818,1.058539,1.066596,1.227814,1.108140,14.83,0.993003,0.994662,0.986191,1.001081,1.130078,1.574916,0.996096,1.009269,1.023029,0.875907,1.070880,1.159008,1.018437,1.003152,0.988173,0.971512,1.040959,0.807246,NaN,NaN,NaN,NaN,NaN,NaN,0.959589,0.996868,1.007342,1.004945,1.143689,2.015068,1.171289,1.358530,147.119995,7.1743
2025-08-15,0.997103,1.000627,1.016507,1.027144,1.274097,1.255538,1.000776,1.010966,1.017039,1.002623,1.183864,1.149888,0.999440,1.010349,1.023084,1.018023,1.058189,1.115306,0.999253,1.013924,1.018180,1.024040,1.201134,1.365214,0.995827,0.999027,0.997228,1.035816,1.186547,1.190069,1.017094,1.015453,1.069761,1.090936,1.284120,1.127577,15.09,1.000240,0.996148,0.986982,1.001050,1.123838,1.563628,1.003024,0.99

In [13]:
load_dotenv()
# Get your FRED API key from the environment variable
fred_api_key = os.getenv("FRED_API_KEY")

# Initialize the Fred object with your API key
fred = Fred(api_key=fred_api_key)
macros_df = pd.DataFrame({'A' : []})

# Dictionary to store data for each ticker
# Real Potential Gross Domestic Product (GDPPOT), Billions of Chained 2012 Dollars, QUARTERLY
# https://fred.stlouisfed.org/series/GDPPOT
gdppot = pdr.DataReader("GDPPOT", "fred", start=start).reset_index()
gdppot['gdppot_us_yoy'] = gdppot.GDPPOT/gdppot.GDPPOT.shift(4)-1
gdppot['gdppot_us_qoq'] = gdppot.GDPPOT/gdppot.GDPPOT.shift(1)-1
gdppot["Quarter"] = gdppot.DATE.dt.to_period('Q').dt.start_time
gdppot_to_merge = gdppot[["Quarter", "gdppot_us_yoy", "gdppot_us_qoq"]]

print(gdppot_to_merge.tail(15))
#print(gdppot.columns)

# # "Core CPI index", MONTHLY
# https://fred.stlouisfed.org/series/CPILFESL
# The "Consumer Price Index for All Urban Consumers: All Items Less Food & Energy"
# is an aggregate of prices paid by urban consumers for a typical basket of goods, excluding food and energy.
# This measurement, known as "Core CPI," is widely used by economists because food and energy have very volatile prices.
cpilfesl = pdr.DataReader("CPILFESL", "fred", start=start).reset_index()
cpilfesl['cpi_core_yoy'] = cpilfesl.CPILFESL/cpilfesl.CPILFESL.shift(12)-1
cpilfesl['cpi_core_mom'] = cpilfesl.CPILFESL/cpilfesl.CPILFESL.shift(1)-1
cpilfesl["Month"] = cpilfesl.DATE.dt.to_period('M').dt.start_time
cpilfesl_to_merge = cpilfesl[["Month", "cpi_core_yoy", "cpi_core_mom"]]

#print(cpilfesl.columns)
print(cpilfesl_to_merge.tail(13) )

# Fed rate https://fred.stlouisfed.org/series/FEDFUNDS
fedfunds = pdr.DataReader("FEDFUNDS", "fred", start=start).reset_index()
fedfunds["Month"] = fedfunds.DATE.dt.to_period('M').dt.start_time
fedfunds.set_index('DATE', inplace=True)
print(fedfunds.tail(10))

caseshiller = pdr.DataReader('CSUSHPISA', "fred", start=start).reset_index()
caseshiller["Month"] = caseshiller.DATE.dt.to_period('M').dt.start_time
caseshiller.set_index('DATE', inplace=True)
print(caseshiller.tail(5))

# https://fred.stlouisfed.org/series/DGS1
dgs1 = pdr.DataReader("DGS1", "fred", start=start).reset_index()
print(dgs1.tail())

# https://fred.stlouisfed.org/series/DGS5
dgs5 = pdr.DataReader("DGS5", "fred", start=start).reset_index()
print(dgs5.tail())

# https://fred.stlouisfed.org/series/DGS5
dgs10 = pdr.DataReader("DGS10", "fred", start=start).reset_index()
print(dgs10.tail())


macros1 = [gdppot_to_merge, cpilfesl_to_merge, fedfunds, caseshiller]
macros2 =[dgs1, dgs5, dgs10]
bonds1 = []
for i, b in enumerate(bonds):
    print(i, b)
    b2 = pdr.DataReader(b, 'fred', start=start).reset_index()
    b2['Month'] = b2.DATE.dt.to_period('M').dt.start_time
    print(b2.tail())
    b2.set_index('DATE', inplace=True)
    bonds1.append(b2)


for i, m2 in enumerate(macros2):  
    print(i, m2)
    if macros_df.empty:
        macros_df = m2
    else:
        macros_df = pd.merge(macros_df, m2, on='DATE', how='inner', validate="one_to_one")
    
    time.sleep(1)
print(macros_df.head())
# gdppot_to_merge is Quarterly (but m2 index is daily)
# macros_df["Quarter"] = macros_df.DATE.dt.to_period('Q').dt.start_time
# macros_df["Month"] = macros_df.DATE.dt.to_period('M').dt.start_time
macros_df["Quarter"] = macros_df.DATE.dt.to_period('Q').dt.start_time


macros_df = pd.merge(
            macros_df,
            gdppot_to_merge,            
            on="Quarter",
            #right_index=True,
            how="left",
            validate="many_to_one",
        )
macros_df["Month"] = macros_df.DATE.dt.to_period('M').dt.start_time
macros_df = pd.merge(
            macros_df,
            cpilfesl_to_merge,
            on="Month",
           # right_index=True,
            how="left",
            validate="many_to_one",
        )

macros_df = pd.merge(
            macros_df,
            fedfunds,
            on="Month",
           # right_index=True,
            how="left",
            validate="many_to_one",
        )
macros_df = pd.merge(
            macros_df,
            caseshiller,
            on="Month",
           # right_index=True,
            how="left",
            validate="many_to_one",
        )



for b1 in bonds1:
    macros_df = pd.merge(
            macros_df,
            b1,
            on="Month",
           # right_index=True,
            how="left",
            validate="many_to_one",
        )
fields = ["cpi_core_yoy",
            "cpi_core_mom",
            "FEDFUNDS",
            "caseshiller"
            "DGS1",
            "DGS5",
            "DGS10",
            "gdppot_us_qoq",
            "gdppot_us_yoy"] + bonds
          
for f in fields: macros_df = macros_df.ffill()


       Quarter  gdppot_us_yoy  gdppot_us_qoq
185 2022-01-01       0.020882       0.005366
186 2022-04-01       0.021403       0.005457
187 2022-07-01       0.021799       0.005539
188 2022-10-01       0.022162       0.005618
189 2023-01-01       0.022384       0.005584
190 2023-04-01       0.022552       0.005623
191 2023-07-01       0.022685       0.005670
192 2023-10-01       0.022787       0.005719
193 2024-01-01       0.022959       0.005753
194 2024-04-01       0.023139       0.005799
195 2024-07-01       0.023223       0.005753
196 2024-10-01       0.023249       0.005745
197 2025-01-01       0.023113       0.005619
198 2025-04-01       0.023029       0.005716
199 2025-07-01       0.023013       0.005737
         Month  cpi_core_yoy  cpi_core_mom
586 2024-07-01      0.032283      0.001853
587 2024-08-01      0.032918      0.002841
588 2024-09-01      0.032899      0.003114
589 2024-10-01      0.032928      0.002659
590 2024-11-01      0.032801      0.002894
591 2024-12-01      0.

In [14]:
macros_df.columns

Index(['DATE', 'DGS1', 'DGS5', 'DGS10', 'Quarter', 'gdppot_us_yoy',
       'gdppot_us_qoq', 'Month', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
       'CSUSHPISA', 'IRLTLT01DEM156N', 'IRLTLT01GBM156N', 'IRLTLT01JPM156N'],
      dtype='object')

In [15]:
macros_df.tail(50)

,DATE,DGS1,DGS5,DGS10,Quarter,gdppot_us_yoy,gdppot_us_qoq,Month,cpi_core_yoy,cpi_core_mom,FEDFUNDS,CSUSHPISA,IRLTLT01DEM156N,IRLTLT01GBM156N,IRLTLT01JPM156N
12995,2025-06-17,4.10,3.99,4.39,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420
12996,2025-06-18,4.10,3.98,4.38,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420
12997,2025-06-19,4.10,3.98,4.38,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420
12998,2025-06-20,4.07,3.96,4.38,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420
12999,2025-06-23,4.00,3.91,4.34,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420
13000,2025-06-24,3.99,3.86,4.30,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420
13001,2025-06-25,3.99,3.83,4.29,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420
13002,2025-06-26,3.96,3.79,4.26,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420
13003,2025-06-27,3.97,3.83,4.29,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420
13004,2025-06-30,3.96,3.79,4.24,2025-04-01,0.023029,0.005716,2025-06-01,0.029079,0.002282,4.33,326.358,2.519048,4.5248,1.420


In [16]:
#save data
#econ_indices_df.reset_index()
names = ["stocks_df", "econ_indices_df", "macros_df"]
data_dir = os.getcwd() + "/data"
for name in names:
    file_name = data_dir + name + '.parquet'
    if os.path.exists(file_name):
        os.remove(file_name)
print(data_dir)        
stocks_df.parquet = stocks_df.to_parquet(os.path.join(data_dir, names[0] + '.parquet'), compression="brotli")
econ_indices_df.parquet = econ_indices_df.to_parquet(os.path.join(data_dir, names[1] + '.parquet'), compression="brotli")
macros_df.parquet = macros_df.to_parquet(os.path.join(data_dir, names[2] + '.parquet'), compression="brotli")

/home/julie/market_analytics/market_analytics_capstone/data


In [14]:

"""Load files from the local directory"""
stocks_df = pd.read_parquet(os.path.join(data_dir, "stocks_df.parquet"))
econ_indices_df = pd.read_parquet(os.path.join(data_dir, "econ_indices_df.parquet"))
macros_df = pd.read_parquet(os.path.join(data_dir, "macros_df.parquet"))